### Import das Bases

In [5]:
import numpy as np
import pandas as pd
import datetime

In [6]:
base = pd.read_csv('df_final_cobertura_historico_updated.csv')

# Import base crunchbase do mês de análise atual
crunhbase_df = pd.read_csv('funding-rounds-29-08-2024.csv')

# Import base domnique atualizada contendo o mês de análise
domo_df = pd.read_csv('domo_all_companies_2024_08_29.csv')

### Formatação de Bases

In [7]:
#Retirando Negócio do nome das startups da base DOMO
domo_df['name'] = domo_df['name'].str.replace('Negócio ', '')

domo_df['name modified'] = domo_df['name'].str.replace(' ', '').str.lower()

#Retirando startups com o stagename Deleted e Url vazia
domo_df = domo_df[domo_df['stagename'] != 'Deleted']
domo_df_url = domo_df[~domo_df['url'].isna()]

In [8]:
# Base Crunchbase
crunhbase_df['Announced Date'] = pd.to_datetime(crunhbase_df['Announced Date'])

# Extract the year and create a new "year" column
crunhbase_df['Year'] = crunhbase_df['Announced Date'].dt.year

# Limpando Url da base do CrunchBase
crunhbase_df['Organization Website'] = crunhbase_df['Organization Website'].str.replace('http://www.', '').str.replace('https://www.', '').str.replace('http://', '').str.replace('https://', '').str.rstrip('/').str.replace('/app', '').str.replace('/en', '')
crunhbase_df['Organization Name Modified'] = crunhbase_df['Organization Name'].str.replace(' ', '').str.lower()

### Merge Base Crunchbase e Domo

In [9]:
# Merge on Left na base Crunchbase, sendo assim as informações de rodada do crunchbase serão prevalentes

selected_columns = domo_df[['id','name modified', 'url', 'stagename']]
selected_columns_url = domo_df_url[['id','name modified', 'url', 'stagename']]

merged_name = pd.merge(crunhbase_df, selected_columns, how='left', left_on=['Organization Name Modified'], right_on=['name modified'])
merged_url = pd.merge(crunhbase_df, selected_columns_url, how='left', left_on=['Organization Website'], right_on=['url'])

merged_data = pd.concat([merged_name, merged_url], ignore_index=True)

In [10]:
# Fazendo limpeza do merge para não ter duplicada de name e url

# Find organizations with non-NaN 'Stagename'
orgs_with_stagename = merged_data.dropna(subset=['stagename'])['Organization Name']

# Filter the DataFrame to remove rows with NaN 'Stagename' for organizations with non-NaN 'Stagename'
merged_data = merged_data[~((merged_data['Organization Name'].isin(orgs_with_stagename)) & (merged_data['stagename'].isnull()))]

merged_data = merged_data.drop_duplicates()

### Merge Base Classificada com Merged_Data acima (Crunchbase - DOMO do mês de análise)

In [11]:
# Supondo que seu DataFrame se chame df
base.rename(columns={'stagename_x': 'stagename'}, inplace=True)

# Filtrando as colunas
base_filtered = base.drop(columns=['Unnamed: 0','Analise descarte', 'stagename_y', 'Diversity Spotlight (US Headquarters Only)'])

In [12]:
# Criando colunas de tempo desde o anúncio da rodada e Month para Merged_data e Base_filtered

today = datetime.date.today()

merged_data['Announced Date'] = pd.to_datetime(merged_data['Announced Date'])
merged_data['Time Since Announced'] = today - merged_data['Announced Date'].dt.date

merged_data['Time Since Announced'] = pd.to_timedelta(merged_data['Time Since Announced'])
merged_data['Time Since Announced'] = merged_data['Time Since Announced'].dt.days

merged_data['Month'] = pd.to_datetime(merged_data['Announced Date']).dt.month

base_filtered['Announced Date'] = pd.to_datetime(base_filtered['Announced Date'])
base_filtered['Time Since Announced'] = today - base_filtered['Announced Date'].dt.date

base_filtered['Time Since Announced'] = pd.to_timedelta(base_filtered['Time Since Announced'])
base_filtered['Time Since Announced'] = base_filtered['Time Since Announced'].dt.days

### Regras de classificação prévia, sem intervenção manual

In [13]:
# Aplicar a condição diretamente para criar a nova coluna 'Classification'

merged_data['Classification'] = np.where(
    merged_data['stagename'] == 'Portfolio', 'Portfolio',
    np.where(
        merged_data['Funding Type'] == 'Post-IPO Debt', 'Provavel Não se aplica',
          np.where(
              merged_data['stagename'].isin(['Cadastro Startup', 'Registered', 'Fazer Teste de Aderência']), 'Nao tivemos acesso',
              np.where(
                  (merged_data['stagename'] == 'Discard') & (merged_data['Total Funding Amount (in USD)'] < 1500000), 'Indefinido',
                  np.where(
                      merged_data['stagename'].isna(), 'Nao tivemos acesso',
                          np.where(
                              (merged_data['stagename'].isin(['Opportunity', 'Zero', 'P.Comitê 01', 'P.Comitê 02'])) & (merged_data['Total Funding Amount (in USD)'] < 1000000), 'Em análise',
                              'Pendente Classificação'  # Default value
                    )
                )
            )
        )
    )
)

In [14]:
merged_data['stagename'] = merged_data['stagename'].fillna('Nao tivemos acesso')

### Verificando se a startup já está na base

In [15]:
# Realizar a junção com base no 'Organization Name'
merged_data = pd.merge( merged_data, base[['Organization Name', 'Classification']],
    on='Organization Name', how='left', suffixes=('', '_base'))

# Preencher a coluna 'Classification' original com valores de 'Classification_base' quando houver correspondência
merged_data['Classification'] = merged_data['Classification_base'].combine_first(merged_data['Classification'])

# Remover a coluna auxiliar 'Classification_base'
merged_data.drop(columns=['Classification_base'], inplace=True)

# Remover Duplicadas
merged_data = merged_data.drop_duplicates(subset=['Organization Name'])

merged_data

,Organization Name,Organization Name URL,Transaction Name,Transaction Name URL,Total Funding Amount,Total Funding Amount Currency,Total Funding Amount (in USD),Funding Type,Organization Industries,Announced Date,...,Organization Location,Year,Organization Name Modified,id,name modified,url,stagename,Time Since Announced,Month,Classification
0,Embraer,https://www.crunchbase.com/organization/embrae...,Post-IPO Debt - Embraer,https://www.crunchbase.com/funding_round/embra...,2.113180e+09,USD,2.113180e+09,Post-IPO Debt,"Aerospace, Commercial, Industrial, Manufacturing",2024-08-06,...,"São Paulo, Sao Paulo, Brazil, South America",2024,embraer,NaN,NaN,NaN,Nao tivemos acesso,23,8,Provavel Não se aplica
1,Magie,https://www.crunchbase.com/organization/magie,Seed Round - Magie,https://www.crunchbase.com/funding_round/magie...,2.800000e+07,BRL,5.194349e+06,Seed,"Information Technology, Internet",2024-08-22,...,"São Paulo, Sao Paulo, Brazil, South America",2024,magie,26228.0,magie,magie.com.br,Fazer Teste de Aderência,7,8,Nao tivemos acesso
3,TEPMED,https://www.crunchbase.com/organization/tepmed,Venture Round - TEPMED,https://www.crunchbase.com/funding_round/tepme...,2.000000e+07,BRL,3.645445e+06,Venture - Series Unknown,"Financial Services, FinTech",2024-08-15,...,"Santa Maria, Rio Grande do Sul, Brazil, South ...",2024,tepmed,NaN,NaN,NaN,Nao tivemos acesso,14,8,Nao tivemos acesso
4,ESG Now,https://www.crunchbase.com/organization/esg-now,Pre Seed Round - ESG Now,https://www.crunchbase.com/funding_round/esg-n...,2.600000e+06,BRL,4.960730e+05,Pre-Seed,"CleanTech, SaaS, Software",2024-08-21,...,"Taquara, Rio Grande do Sul, Brazil, South America",2024,esgnow,20428.0,esgnow,esgnow.co,Portfolio,8,8,Portfolio
6,Afya,https://www.crunchbase.com/organization/afya-e...,Post-IPO Debt - Afya,https://www.crunchbase.com/funding_round/afya-...,2.388841e+08,USD,2.388841e+08,Post-IPO Debt,"EdTech, Education, Health Care, Higher Educati...",2024-08-07,...,"Minas Gerais, Acre, Brazil, South America",2024,afya,NaN,NaN,NaN,Nao tivemos acesso,22,8,Provavel Não se aplica
7,Agrion Agrisolutions,https://www.crunchbase.com/organization/agrion...,Venture Round - Agrion Agrisolutions,https://www.crunchbase.com/funding_round/agrio...,5.000000e+07,USD,5.000000e+07,Venture - Series Unknown,"AgTech, Biotechnology, Fertility",2024-08-27,...,"Uberlândia, Minas Gerais, Brazil, South America",2024,agrionagrisolutions,NaN,NaN,NaN,Nao tivemos acesso,2,8,Não se aplica
8,Lavoro Agro,https://www.crunchbase.com/organization/lavoro...,Post-IPO Debt - Lavoro Agro,https://www.crunchbase.com/funding_round/lavor...,7.300000e+08,BRL,1.413400e+08,Post-IPO Debt,"Agriculture, Productivity Tools",2024-08-02,...,"Munhoz De Melo, Parana, Brazil, South America",2024,lavoroagro,NaN,NaN,NaN,Nao tivemos acesso,27,8,Não se aplica
10,Scala Data Centers,https://www.crunchbase.com/organization/scala-...,Debt Financing - Scala Data Centers,https://www.crunchbase.com/funding_round/scala...,4.540566e+08,USD,4.540566e+08,Debt Financing,"Data Center, Data Storage, Information Technology",2024-08-20,...,"São Paulo, Sao Paulo, Brazil, South America",2024,scaladatacenters,NaN,NaN,NaN,Nao tivemos acesso,9,8,Nao tivemos acesso
11,Cedro Textil,https://www.crunchbase.com/organization/cedro-...,Post-IPO Debt - Cedro Textil,https://www.crunchbase.com/funding_round/cedro...,NaN,NaN,NaN,Post-IPO Debt,"Fashion, Textiles",2024-08-02,...,"Belo Horizonte, Minas Gerais, Brazil, South Am...",2024,cedrotextil,NaN,NaN,NaN,Nao tivemos acesso,27,8,Não se aplica
12,Cayena,https://www.crunchbase.com/organization/poupachef,Series B - Cayena,https://www.crunchbase.com/funding_round/poupa...,3.400000e+07,USD,3.400000e+07,Series B,"B2B, E-Commerce, Food and Beverage, Informatio...",2024-07-30,...,"São Paulo, Sao Paulo, Brazil, South America",2024,cayena,19457.0,cayena,cayena.com,Discard,30,7,Falso Negativo


### Unindo base histórica e atualização do mês

In [16]:
# Supondo que merged_data e base_filtered sejam os DataFrames que você quer concatenar
combined_data = pd.concat([merged_data, base_filtered])

# Removendo duplicatas
combined_data = combined_data.drop_duplicates(subset=['Organization Name', 'Announced Date', 'stagename'])

In [17]:
# Convertendo 'Announced Date' para o tipo datetime
combined_data['Announced Date'] = pd.to_datetime(combined_data['Announced Date'], errors='coerce')

# Ordenando pelo 'Announced Date'
combined_data = combined_data.sort_values(by='Announced Date')
combined_data = combined_data.reset_index(drop=True)

In [18]:
combined_data = combined_data[combined_data['Classification'] != 'Não se aplica']

In [20]:
combined_data.to_csv('df_final_cobertura_historico.csv', index=False)